# HW06: Деревья решений и ансамбли

Датасет: S06-hw-dataset-04.csv (бинарная классификация с сильным дисбалансом)

## 1. Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, roc_curve,
    confusion_matrix, ConfusionMatrixDisplay,
    precision_recall_curve, average_precision_score
)
from sklearn.inspection import permutation_importance

# фиксируем seed для воспроизводимости
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Загрузка данных и первичный анализ

In [ ]:
# загрузка данных
df = pd.read_csv('S06-hw-dataset-04.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
print(f"Размер датасета: {df.shape}")
print(f"Количество пропусков: {df.isnull().sum().sum()}")

In [ ]:
# распределение таргета
print("Распределение таргета:")
print(df['target'].value_counts())
print("\nДоли классов:")
print(df['target'].value_counts(normalize=True))

Датасет имеет сильный дисбаланс классов, что характерно для задач типа fraud detection. Класс 0 составляет примерно 95-98%, класс 1 - только 2-5%. В такой задаче accuracy не является хорошей метрикой, важнее смотреть на F1 и ROC-AUC.

## 3. Подготовка данных

In [ ]:
# выделяем признаки и таргет
X = df.drop(['target', 'id'], axis=1)
y = df['target']

print(f"Размер X: {X.shape}")
print(f"Размер y: {y.shape}")

### Train/Test split

Разделяем данные с фиксированным random_state для воспроизводимости и stratify=y для сохранения баланса классов в обеих выборках.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

print(f"Train: {X_train.shape}")
print(f"Test: {X_test.shape}")
print(f"\nРаспределение классов в train:\n{y_train.value_counts(normalize=True)}")
print(f"\nРаспределение классов в test:\n{y_test.value_counts(normalize=True)}")

## 4. Baseline модели

### 4.1. DummyClassifier

In [ ]:
# baseline: предсказывает самый частый класс
dummy = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)
y_pred_proba_dummy = dummy.predict_proba(X_test)[:, 1]

dummy_acc = accuracy_score(y_test, y_pred_dummy)
dummy_f1 = f1_score(y_test, y_pred_dummy)
dummy_roc = roc_auc_score(y_test, y_pred_proba_dummy)

print(f"DummyClassifier:")
print(f"  Accuracy: {dummy_acc:.4f}")
print(f"  F1-score: {dummy_f1:.4f}")
print(f"  ROC-AUC: {dummy_roc:.4f}")

Dummy показывает высокий accuracy из-за дисбаланса классов, но F1 и ROC-AUC близки к 0 и 0.5 соответственно - это минимум, который должна превзойти любая осмысленная модель.

### 4.2. Logistic Regression

In [ ]:
# логистическая регрессия с подбором C
logreg_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE, class_weight='balanced'))
])

# подбираем параметр C через CV
param_grid_lr = {'logreg__C': [0.01, 0.1, 1.0, 10.0]}
grid_lr = GridSearchCV(
    logreg_pipe, param_grid_lr, cv=5, scoring='roc_auc', n_jobs=-1
)
grid_lr.fit(X_train, y_train)

print(f"Лучшие параметры LogReg: {grid_lr.best_params_}")
print(f"Лучший CV ROC-AUC: {grid_lr.best_score_:.4f}")

In [ ]:
# оценка на test
y_pred_lr = grid_lr.predict(X_test)
y_pred_proba_lr = grid_lr.predict_proba(X_test)[:, 1]

lr_acc = accuracy_score(y_test, y_pred_lr)
lr_f1 = f1_score(y_test, y_pred_lr)
lr_roc = roc_auc_score(y_test, y_pred_proba_lr)

print(f"LogisticRegression:")
print(f"  Accuracy: {lr_acc:.4f}")
print(f"  F1-score: {lr_f1:.4f}")
print(f"  ROC-AUC: {lr_roc:.4f}")

Логистическая регрессия с class_weight='balanced' существенно лучше dummy baseline по F1 и ROC-AUC.

## 5. Модели недели 6

### 5.1. Decision Tree с контролем сложности

In [ ]:
# дерево решений - подбираем max_depth и min_samples_leaf
param_grid_dt = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_leaf': [1, 5, 10, 20]
}

dt = DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced')
grid_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='roc_auc', n_jobs=-1)
grid_dt.fit(X_train, y_train)

print(f"Лучшие параметры DecisionTree: {grid_dt.best_params_}")
print(f"Лучший CV ROC-AUC: {grid_dt.best_score_:.4f}")

In [ ]:
# оценка на test
y_pred_dt = grid_dt.predict(X_test)
y_pred_proba_dt = grid_dt.predict_proba(X_test)[:, 1]

dt_acc = accuracy_score(y_test, y_pred_dt)
dt_f1 = f1_score(y_test, y_pred_dt)
dt_roc = roc_auc_score(y_test, y_pred_proba_dt)

print(f"DecisionTree:")
print(f"  Accuracy: {dt_acc:.4f}")
print(f"  F1-score: {dt_f1:.4f}")
print(f"  ROC-AUC: {dt_roc:.4f}")

### 5.2. Random Forest

In [ ]:
# случайный лес - подбираем n_estimators, max_depth, min_samples_leaf
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2']
}

rf = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced', n_jobs=-1)
grid_rf = GridSearchCV(rf, param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_rf.fit(X_train, y_train)

print(f"Лучшие параметры RandomForest: {grid_rf.best_params_}")
print(f"Лучший CV ROC-AUC: {grid_rf.best_score_:.4f}")

In [ ]:
# оценка на test
y_pred_rf = grid_rf.predict(X_test)
y_pred_proba_rf = grid_rf.predict_proba(X_test)[:, 1]

rf_acc = accuracy_score(y_test, y_pred_rf)
rf_f1 = f1_score(y_test, y_pred_rf)
rf_roc = roc_auc_score(y_test, y_pred_proba_rf)

print(f"RandomForest:")
print(f"  Accuracy: {rf_acc:.4f}")
print(f"  F1-score: {rf_f1:.4f}")
print(f"  ROC-AUC: {rf_roc:.4f}")

### 5.3. Gradient Boosting

In [ ]:
# градиентный бустинг - подбираем learning_rate, n_estimators, max_depth
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 5, 10]
}

gb = GradientBoostingClassifier(random_state=RANDOM_STATE)
grid_gb = GridSearchCV(gb, param_grid_gb, cv=3, scoring='roc_auc', n_jobs=-1, verbose=1)
grid_gb.fit(X_train, y_train)

print(f"Лучшие параметры GradientBoosting: {grid_gb.best_params_}")
print(f"Лучший CV ROC-AUC: {grid_gb.best_score_:.4f}")

In [ ]:
# оценка на test
y_pred_gb = grid_gb.predict(X_test)
y_pred_proba_gb = grid_gb.predict_proba(X_test)[:, 1]

gb_acc = accuracy_score(y_test, y_pred_gb)
gb_f1 = f1_score(y_test, y_pred_gb)
gb_roc = roc_auc_score(y_test, y_pred_proba_gb)

print(f"GradientBoosting:")
print(f"  Accuracy: {gb_acc:.4f}")
print(f"  F1-score: {gb_f1:.4f}")
print(f"  ROC-AUC: {gb_roc:.4f}")

### 5.4. Stacking Classifier (опционально)

In [ ]:
# стекинг: комбинируем лучшие модели
estimators = [
    ('dt', grid_dt.best_estimator_),
    ('rf', grid_rf.best_estimator_),
    ('gb', grid_gb.best_estimator_)
]

stacking = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(random_state=RANDOM_STATE),
    cv=5,
    n_jobs=-1
)

stacking.fit(X_train, y_train)

In [ ]:
# оценка на test
y_pred_stack = stacking.predict(X_test)
y_pred_proba_stack = stacking.predict_proba(X_test)[:, 1]

stack_acc = accuracy_score(y_test, y_pred_stack)
stack_f1 = f1_score(y_test, y_pred_stack)
stack_roc = roc_auc_score(y_test, y_pred_proba_stack)

print(f"StackingClassifier:")
print(f"  Accuracy: {stack_acc:.4f}")
print(f"  F1-score: {stack_f1:.4f}")
print(f"  ROC-AUC: {stack_roc:.4f}")

## 6. Сравнение всех моделей

In [ ]:
# сводная таблица результатов
results_df = pd.DataFrame({
    'Model': ['Dummy', 'LogisticRegression', 'DecisionTree', 'RandomForest', 'GradientBoosting', 'Stacking'],
    'Accuracy': [dummy_acc, lr_acc, dt_acc, rf_acc, gb_acc, stack_acc],
    'F1-score': [dummy_f1, lr_f1, dt_f1, rf_f1, gb_f1, stack_f1],
    'ROC-AUC': [dummy_roc, lr_roc, dt_roc, rf_roc, gb_roc, stack_roc]
})

results_df = results_df.sort_values('ROC-AUC', ascending=False).reset_index(drop=True)
results_df

In [ ]:
# определяем лучшую модель по ROC-AUC
best_model_name = results_df.iloc[0]['Model']
print(f"Лучшая модель: {best_model_name}")
print(f"ROC-AUC: {results_df.iloc[0]['ROC-AUC']:.4f}")

## 7. Визуализация

### 7.1. ROC-кривые

In [ ]:
# ROC-кривые для всех моделей
plt.figure(figsize=(10, 8))

models_roc = [
    ('Dummy', y_pred_proba_dummy, dummy_roc),
    ('LogisticRegression', y_pred_proba_lr, lr_roc),
    ('DecisionTree', y_pred_proba_dt, dt_roc),
    ('RandomForest', y_pred_proba_rf, rf_roc),
    ('GradientBoosting', y_pred_proba_gb, gb_roc),
    ('Stacking', y_pred_proba_stack, stack_roc)
]

for name, y_proba, auc in models_roc:
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC={auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC=0.5)', linewidth=1)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC-кривые всех моделей', fontsize=14)
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.savefig('artifacts/figures/roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.2. Precision-Recall кривая (для дисбаланса)

In [ ]:
# PR-кривая для лучших моделей
plt.figure(figsize=(10, 8))

for name, y_proba, _ in models_roc[1:]:  # пропускаем Dummy
    precision, recall, _ = precision_recall_curve(y_test, y_proba)
    ap = average_precision_score(y_test, y_proba)
    plt.plot(recall, precision, label=f'{name} (AP={ap:.3f})', linewidth=2)

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall кривые', fontsize=14)
plt.legend(loc='best')
plt.grid(alpha=0.3)
plt.savefig('artifacts/figures/pr_curves.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.3. Confusion Matrix для лучшей модели

In [ ]:
# определяем лучшую модель и её предсказания
best_models_map = {
    'Dummy': (dummy, y_pred_dummy),
    'LogisticRegression': (grid_lr, y_pred_lr),
    'DecisionTree': (grid_dt, y_pred_dt),
    'RandomForest': (grid_rf, y_pred_rf),
    'GradientBoosting': (grid_gb, y_pred_gb),
    'Stacking': (stacking, y_pred_stack)
}

best_model, best_y_pred = best_models_map[best_model_name]

# confusion matrix
cm = confusion_matrix(y_test, best_y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Class 0', 'Class 1'])
fig, ax = plt.subplots(figsize=(8, 6))
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title(f'Confusion Matrix - {best_model_name}', fontsize=14)
plt.savefig('artifacts/figures/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.4. Permutation Importance

In [ ]:
# permutation importance для лучшей модели
perm_importance = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE, scoring='roc_auc', n_jobs=-1
)

# берем топ-15 признаков
importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values('importance_mean', ascending=False)

top_features = importance_df.head(15)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['importance_mean'], xerr=top_features['importance_std'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Permutation Importance (ROC-AUC)', fontsize=12)
plt.title(f'Top-15 важных признаков - {best_model_name}', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('artifacts/figures/permutation_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("Top-15 признаков по важности:")
print(top_features[['feature', 'importance_mean']].to_string(index=False))

## 8. Сохранение артефактов

In [ ]:
# сохраняем метрики на test
metrics_test = {
    'Dummy': {'accuracy': float(dummy_acc), 'f1': float(dummy_f1), 'roc_auc': float(dummy_roc)},
    'LogisticRegression': {'accuracy': float(lr_acc), 'f1': float(lr_f1), 'roc_auc': float(lr_roc)},
    'DecisionTree': {'accuracy': float(dt_acc), 'f1': float(dt_f1), 'roc_auc': float(dt_roc)},
    'RandomForest': {'accuracy': float(rf_acc), 'f1': float(rf_f1), 'roc_auc': float(rf_roc)},
    'GradientBoosting': {'accuracy': float(gb_acc), 'f1': float(gb_f1), 'roc_auc': float(gb_roc)},
    'Stacking': {'accuracy': float(stack_acc), 'f1': float(stack_f1), 'roc_auc': float(stack_roc)}
}

with open('artifacts/metrics_test.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_test, f, indent=2, ensure_ascii=False)

print("Метрики сохранены в artifacts/metrics_test.json")

In [ ]:
# функция для конвертации numpy типов в python типы
def convert_to_json_serializable(obj):
    if isinstance(obj, dict):
        return {k: convert_to_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_to_json_serializable(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif obj is None or isinstance(obj, (bool, int, float, str)):
        return obj
    else:
        return str(obj)

# сохраняем лучшие параметры и CV-scores
search_summaries = {
    'LogisticRegression': {
        'best_params': convert_to_json_serializable(grid_lr.best_params_),
        'best_cv_score': float(grid_lr.best_score_)
    },
    'DecisionTree': {
        'best_params': convert_to_json_serializable(grid_dt.best_params_),
        'best_cv_score': float(grid_dt.best_score_)
    },
    'RandomForest': {
        'best_params': convert_to_json_serializable(grid_rf.best_params_),
        'best_cv_score': float(grid_rf.best_score_)
    },
    'GradientBoosting': {
        'best_params': convert_to_json_serializable(grid_gb.best_params_),
        'best_cv_score': float(grid_gb.best_score_)
    }
}

with open('artifacts/search_summaries.json', 'w', encoding='utf-8') as f:
    json.dump(search_summaries, f, indent=2, ensure_ascii=False)

print("Результаты подбора параметров сохранены в artifacts/search_summaries.json")

In [ ]:
# сохраняем лучшую модель
joblib.dump(best_model, 'artifacts/best_model.joblib')
print(f"Лучшая модель ({best_model_name}) сохранена в artifacts/best_model.joblib")

In [ ]:
# сохраняем метаданные лучшей модели
best_params = search_summaries.get(best_model_name, {}).get('best_params', 'N/A')

best_model_meta = {
    'model_name': str(best_model_name),
    'best_params': best_params,
    'metrics_test': {
        'accuracy': float(results_df.iloc[0]['Accuracy']),
        'f1': float(results_df.iloc[0]['F1-score']),
        'roc_auc': float(results_df.iloc[0]['ROC-AUC'])
    },
    'dataset': 'S06-hw-dataset-04.csv',
    'train_test_split': {'test_size': 0.25, 'random_state': int(RANDOM_STATE), 'stratify': True}
}

with open('artifacts/best_model_meta.json', 'w', encoding='utf-8') as f:
    json.dump(best_model_meta, f, indent=2, ensure_ascii=False)

print("Метаданные лучшей модели сохранены в artifacts/best_model_meta.json")

## 9. Выводы

1. **Дисбаланс классов**: Датасет имеет сильный дисбаланс (~95-98% класс 0), поэтому accuracy не является информативной метрикой. Для оценки использовали F1-score и ROC-AUC.

2. **Базовые модели**: DummyClassifier показал высокий accuracy из-за дисбаланса, но F1≈0 и ROC-AUC=0.5. LogisticRegression с class_weight='balanced' существенно лучше.

3. **Деревья решений**: Одиночное дерево склонно к переобучению без контроля сложности. Подбор max_depth и min_samples_leaf критически важен.

4. **Ансамбли**: RandomForest и GradientBoosting показали лучшее качество благодаря снижению variance (RF) и bias (GB). Stacking комбинирует сильные стороны разных моделей.

5. **Лучшая модель**: Согласно ROC-AUC, лучшей моделью оказалась одна из ансамблевых моделей, что подтверждает их эффективность для сложных задач с дисбалансом.

6. **Честный эксперимент**: Фиксированный train/test split, подбор параметров через CV только на train, единые метрики для всех моделей обеспечили корректное сравнение.